<a href="https://colab.research.google.com/github/davidguzmanr/CSC2516/blob/main/text-classifier/Training-BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
from google.colab import runtime

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
! git clone https://github.com/davidguzmanr/CSC2516.git

Cloning into 'CSC2516'...
remote: Enumerating objects: 85, done.
remote: Counting objects: 100% (85/85), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 85 (delta 41), reused 48 (delta 16), pack-reused 0
Unpacking objects: 100% (85/85), 43.49 KiB | 1.55 MiB/s, done.


In [3]:
%cd CSC2516/ 

/content/CSC2516


In [4]:
! pip install -r requirements.txt --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 38.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 KB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.0/159.0 KB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 KB 37.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.9/216.9 KB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 708.1/708.1 KB 58.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 54.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.4/887.4 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 529.2/529.2 KB 46.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 82.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.2/24.2 MB 72.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 110.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
%cd text-classifier/ 

In [6]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [ ]:
%tensorboard --logdir=lightning_logs

# BERT

In [8]:
from datasets import load_dataset

load_dataset('rotten_tomatoes')

Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Dataset rotten_tomatoes downloaded and prepared to /root/.cache/huggingface/datasets/rotten_tomatoes/default/1.0.0/40d411e45a6ce3484deed7cc15b82a53dad9a72aafd9f86f8f227134bec5ca46. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 8530
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
})

## Model=bert-base-cased, optimizer=SGD, with weight decay, $\varepsilon = [10, 30, 50]$

See [How do you run multiple programs in parallel from a bash script?](https://stackoverflow.com/questions/3004811/how-do-you-run-multiple-programs-in-parallel-from-a-bash-script)

In [ ]:
! python train.py --config=config.yaml --model.differential_privacy=False --model.data_augmentation=False & python train.py --config=config.yaml --model.differential_privacy=False --model.data_augmentation=True && fg

In [ ]:
# With DP, epsilon=10
! python train.py --config=config.yaml --model.differential_privacy=True --model.epsilon=10.0 --model.data_augmentation=False & python train.py --config=config.yaml --model.differential_privacy=True --model.epsilon=10.0 --model.data_augmentation=True && fg

In [ ]:
# With DP, epsilon=30
! python train.py --config=config.yaml --model.differential_privacy=True --model.epsilon=30.0 --model.data_augmentation=False & python train.py --config=config.yaml --model.differential_privacy=True --model.epsilon=30.0 --model.data_augmentation=True && fg

In [ ]:
# With DP, epsilon=50
! python train.py --config=config.yaml --model.differential_privacy=True --model.epsilon=50.0 --model.data_augmentation=False & python train.py --config=config.yaml --model.differential_privacy=True --model.epsilon=50.0 --model.data_augmentation=True && fg

In [ ]:
! zip -r bert-sgd-with-regularization.zip lightning_logs/
! cp "/content/CSC2516/text-classifier/bert-sgd-with-regularization.zip" "/content/gdrive/MyDrive/bert-sgd-with-regularization.zip"
! rm -r lightning_logs/

## Model=Resnet18, optimizer=SGD, no weight decay, $\varepsilon = [10, 30, 50]$

See [How do you run multiple programs in parallel from a bash script?](https://stackoverflow.com/questions/3004811/how-do-you-run-multiple-programs-in-parallel-from-a-bash-script)

In [ ]:
! python train.py --config=config.yaml --model.weight_decay=0.0 --model.differential_privacy=False --model.data_augmentation=False & python train.py --config=config.yaml --model.weight_decay=0.0 --model.differential_privacy=False --model.data_augmentation=True && fg

In [ ]:
# With DP, epsilon=10
! python train.py --config=config.yaml --model.weight_decay=0.0 --model.differential_privacy=True --model.epsilon=10.0 --model.data_augmentation=False & python train.py --config=config.yaml --model.weight_decay=0.0 --model.differential_privacy=True --model.epsilon=10.0 --model.data_augmentation=True && fg

In [ ]:
# With DP, epsilon=30
! python train.py --config=config.yaml --model.weight_decay=0.0 --model.differential_privacy=True --model.epsilon=30.0 --model.data_augmentation=False & python train.py --config=config.yaml --model.weight_decay=0.0 --model.differential_privacy=True --model.epsilon=30.0 --model.data_augmentation=True && fg

In [ ]:
# With DP, epsilon=50
! python train.py --config=config.yaml --model.weight_decay=0.0 --model.differential_privacy=True --model.epsilon=50.0 --model.data_augmentation=False & python train.py --config=config.yaml --model.weight_decay=0.0 --model.differential_privacy=True --model.epsilon=50.0 --model.data_augmentation=True && fg

In [ ]:
! zip -r bert-sgd-no-regularization.zip lightning_logs/
! cp "/content/CSC2516/text-classifier/bert-sgd-no-regularization.zip" "/content/gdrive/MyDrive/bert-sgd-no-regularization.zip"
! rm -r lightning_logs/

In [ ]:
# runtime.unassign()